# Lecture 10
## Object Oriented Programming II
### Wednesday, October 4th 2017

In [1]:
from IPython.display import HTML

## Recap
* We introduced the idea of objects in Python
* We discussed Python classes
* We went over **inheritance**
* We *briefly* touched on **polymorphism**
* We started to introduce Python special methods (the *dunder* methods)

## Example:  Printing with `__repr__` and `__str__`

* The way printing works is that Python wants classes to implement `__repr__` and `__str__` methods. 
* It will use inheritance to give the built-in `object`s methods when these are not defined.
* Any class can define `__repr__` and `__str__`. 
* When an *instance* of such a class is interrogated with the `repr` or `str` function, then these underlying methods are called.

We'll see `__repr__` here. If you define `__repr__` you have made an object sensibly printable.

####   `__repr__`  

In [2]:
class Animal():
    
    def __init__(self, name):
        self.name=name
        
    def __repr__(self):
        class_name = type(self).__name__
        return "{0!s}({1.name!r})".format(class_name, self)

In [3]:
r = Animal("David")
r

Animal('David')

In [4]:
print(r)

Animal('David')


In [5]:
repr(r)

"Animal('David')"

#### Notes on `__repr__`
* The return value of `__repr__` is in quotes.  Why?
* The expression returned by `__repr__` should be able to be fed into the `eval` built-in.
  - `eval` accepts a `Python` expression as a string.
  - The `Python` expression is then evaluated.
  - Convenient for debugging!
* `__repr__` returns the `Python` code needed to rebuild our object.

In [6]:
eval(repr(r))

Animal('David')

Now we see how `r` was created!

### The pattern with *dunder* methods


**There are functions without double-underscores that cause the methods with the double-underscores to be called**

Thus `repr(an_object)` will cause `an_object.__repr__()` to be called. 

In user-level code, you *SHOULD NEVER* see the latter. In library level code, you might see the latter. The definition of the class is considered library level code.

## Example:  Instance Equality via `__eq__`

We can now ask and answer the question:  What makes two objects equal?

To do this, we will add a new dunder method to the mix, the unimaginatively named (that's a good thing) `__eq__`.

In [7]:
class Animal():
    
    def __init__(self, name):
        self.name=name
        
    def __repr__(self):
        class_name = type(self).__name__
        return "{0!s}({1.name!r})".format(class_name, self)
    
    def __eq__(self, other):
        return self.name==other.name # two animals are equal if their names are equal

In [8]:
A=Animal("Tom")
B=Animal("Jane")
C=Animal("Tom")

There are three separate object identities, but we made two of them equal!

In [9]:
print(id(A), "   ", id(B), "   ", id(C))

print(A==B, "         ", B==C, "         ", A==C)

4458697560     4458697504     4458697616
False           False           True


This is critical because it gives us a say in what equality means.

## Python's power comes from the data model, composition, and delegation

The data model is used (from **Fluent Python**) to provide a:

>description of the interfaces of the building blocks of the language itself, such as sequences, iterators, functions, classes....

The special "dunder" methods we talk about are invoked by the `Python` interpreter to perform basic operations. 

For example, `__getitem__` gets an item in a sequence. This is used to do something like `a[3]`. 

`__len__` is used to say how long a sequence is. Its invoked by the `len` built-in function. 

A **sequence**, for example,  must implement `__len__` and `__getitem__`. Thats it.

The original reference for this data model is: https://docs.python.org/3/reference/datamodel.html .

### Tuple

An example of a sequence in Python is the tuple. Since a tuple is a sequence, it must support indexing and be able to tell us its length.

In [10]:
a=(1,2)
a[0]

1

In [11]:
len(a)

2

### NamedTuples

#### `collections.namedtuple`
* Produces subclasses of tuples
* The tuples are enhanced with field names and a class name.

Consider the example from **Fluent Python** (Example 1-1):

In [12]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])
repr(Card)

"<class '__main__.Card'>"

In [13]:
my_card = Card('3', 'diamonds')
print(my_card)
print(type(my_card))
print(my_card.rank)

Card(rank='3', suit='diamonds')
<class '__main__.Card'>
3


#### A Custom Sequence

We now wish to create a `FrenchDeck` as an example of something that follows Python's Sequence protocol. Remember, the sequence protocol requires implementation of two methods: `__len__` and `__getitem__`. Thats it.

In [14]:
class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JKQA')
    suits="spade diamond club heart".split()
    
    def __init__(self):
        # composition: there are items IN this class that constutute its structure
        # delegation: the storage for this class is DELEGATED to this list below
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [15]:
class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JKQA')
    suits="spade diamond club heart".split()
    
    def __init__(self):
        # composition: there are items IN this class that constutute its structure
        # delegation: the storage for this class is DELEGATED to this list below
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [16]:
deck = FrenchDeck()
len(deck)

52

In [17]:
deck[0], deck[-1], deck[3]

(Card(rank='2', suit='spade'),
 Card(rank='A', suit='heart'),
 Card(rank='5', suit='spade'))

In [18]:
deck[10:15]

[Card(rank='K', suit='spade'),
 Card(rank='Q', suit='spade'),
 Card(rank='A', suit='spade'),
 Card(rank='2', suit='diamond'),
 Card(rank='3', suit='diamond')]

* The `FrenchDeck` class supports the sequence protocol
* As a result, we can use functions like `random.choice` *directly* on instances of `FrenchDeck`. 
* This is the power of interfaces and the data model.

In [19]:
from random import choice
choice(deck)

Card(rank='7', suit='diamond')

### Building out our class: instances and classmethods

In [20]:
class ComplexClass():
    def __init__(self, a, b):
        self.real = a
        self.imaginary = b
        
    @classmethod
    def make_complex(cls, a, b):
        return cls(a, b)
        
    def __repr__(self):
        class_name = type(self).__name__
        return "%s(real=%r, imaginary=%r)" % (class_name, self.real, self.imaginary)
        
    def __eq__(self, other):
        return (self.real == other.real) and (self.imaginary == other.imaginary)

In [21]:
c1 = ComplexClass(1,2)
c1

ComplexClass(real=1, imaginary=2)

`make_complex` is a class method. See how its signature is different above. It is a factory to produce instances.

In [22]:
c2 = ComplexClass.make_complex(1,2)
c2

ComplexClass(real=1, imaginary=2)

In [23]:
c1 == c2

True

Wouldn't it be great to define other operations (like addition and subtraction)?

### Static Methods, Class Methods, Instance Methods

What's really going on under the hood here?

In [24]:
# From fluent python
class Demo():
    @classmethod
    def klassmeth(*args): # Class methods do not have to return an instance of the class
        return args
    
    @staticmethod
    def statmeth(*args): # This is just a regular function
        return args
    
    def instmeth(*args): # This is a true blue instance method
        return args
    

In [25]:
notademo = Demo.statmeth(1,2)
print(type(notademo))
notademo

<class 'tuple'>


(1, 2)

In [26]:
ademo = Demo.klassmeth(1,2)
print(type(ademo))
ademo

<class 'tuple'>


(__main__.Demo, 1, 2)

In [27]:
ademo = Demo()
Demo.instmeth(ademo, 1,2)

(<__main__.Demo at 0x109c9b4e0>, 1, 2)

In [28]:
ademo.instmeth(1,2)

(<__main__.Demo at 0x109c9b4e0>, 1, 2)

[PythonTutor Example](https://goo.gl/Q9UNK2)

### Class variables and instance variables



In [29]:
class Demo2():
    classvar=1
      
ademo2 = Demo2()
print(Demo2.classvar, ademo2.classvar)
ademo2.classvar=2 # Different from the classvar above
print(Demo2.classvar, ademo2.classvar)

1 1
1 2


[PythonTutor Example](https://goo.gl/3HnEGZ)

## Code and Data for objects

In [30]:
class A(object):
    
    def __init__(self, x):
        self.x = x
        
    def doit(self, y):
        return self.x + y

`dir` for classes contains the names of its attributes, and recursively of the attributes of its bases.

In [31]:
dir(A)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'doit']

`vars` on an object gets the contents of a special attribute called `__dict__`.

In [32]:
vars(A)

mappingproxy({'__dict__': <attribute '__dict__' of 'A' objects>,
              '__doc__': None,
              '__init__': <function __main__.A.__init__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              'doit': <function __main__.A.doit>})

Let's make an instance of `A`.

In [33]:
a=A(5)

`dir` again:

In [34]:
dir(a)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'doit',
 'x']

`var` again:

In [35]:
vars(a)

{'x': 5}

There is some kind of a table implementation for Python objects (it's written in C). 

This implementation allows us to look for attributes and methods, and if not found look elsewhere. 

The exact details are complex, using descriptors and other lookups, and we'll tackle them in more detail later (hopefully). 

ut currently it suffices us to know that lookup first happens in the instance table, followed by the class table (methods) and if not there somewhere up in the inheritance hierarchy.

In [36]:
A.__class__, a.__class__

(type, __main__.A)

# What is Polymorphism?

We saw polymorphism last time as the ability to tun the same methods on different objects, either through inheritance or by just defining an ad-hoc protocol (duck typing).

The more general definition is:

**The ability to write code that looks similar, but operates on different types.**

In other words, a single interface serves entities of different types. 


## Polymorphism Summary

Python type system is strong and dynamic:

- strong: everything has a well-defined type: `type`, `isinstance`
- dynamic: type is not explicitly declared, changes with content

In classic dynamically typed languages (e.g. Python) most common code is polymorphic. 

The types of values are restricted only by explicit runtime checks or errors due to failed support for operations at run time. 

Polymorphism is often combined with inheritance, but does not need to be.

One classification of Polymorphism (summarized from Wikipedia) divides it as:
- on one axis:  adhoc, parametric, and subtype based
- on another axis:  dynamic (run time) and static (compile time)

### Static vs Dynamic

Dynamic (run-time) polymorphism can be thought of as table based dispatch: that there is, somewhere, atleast conceptually, a table of types, or a linkage of such tables created by inheritance, where implementations are looked up for types.

In static polymorphism (e.g. in `C++`) the binding to the appropriate class can be done at compile time.

### Back to our French Deck Example

In [37]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])

In [38]:
class FrenchDeck():
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
                       
    def __len__(self):
        return len(self._cards)
                       
    def __getitem__(self, position): 
        return self._cards[position]
        

In [39]:
mydeck = FrenchDeck()
print(len(mydeck))

52


In [40]:
vars(mydeck)

{'_cards': [Card(rank='2', suit='spades'),
  Card(rank='3', suit='spades'),
  Card(rank='4', suit='spades'),
  Card(rank='5', suit='spades'),
  Card(rank='6', suit='spades'),
  Card(rank='7', suit='spades'),
  Card(rank='8', suit='spades'),
  Card(rank='9', suit='spades'),
  Card(rank='10', suit='spades'),
  Card(rank='J', suit='spades'),
  Card(rank='Q', suit='spades'),
  Card(rank='K', suit='spades'),
  Card(rank='A', suit='spades'),
  Card(rank='2', suit='diamonds'),
  Card(rank='3', suit='diamonds'),
  Card(rank='4', suit='diamonds'),
  Card(rank='5', suit='diamonds'),
  Card(rank='6', suit='diamonds'),
  Card(rank='7', suit='diamonds'),
  Card(rank='8', suit='diamonds'),
  Card(rank='9', suit='diamonds'),
  Card(rank='10', suit='diamonds'),
  Card(rank='J', suit='diamonds'),
  Card(rank='Q', suit='diamonds'),
  Card(rank='K', suit='diamonds'),
  Card(rank='A', suit='diamonds'),
  Card(rank='2', suit='clubs'),
  Card(rank='3', suit='clubs'),
  Card(rank='4', suit='clubs'),
  Card(r

In [41]:
dir(mydeck)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cards',
 'ranks',
 'suits']

In [42]:
vars(mydeck.__class__)

mappingproxy({'__dict__': <attribute '__dict__' of 'FrenchDeck' objects>,
              '__doc__': None,
              '__getitem__': <function __main__.FrenchDeck.__getitem__>,
              '__init__': <function __main__.FrenchDeck.__init__>,
              '__len__': <function __main__.FrenchDeck.__len__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'FrenchDeck' objects>,
              'ranks': ['2',
               '3',
               '4',
               '5',
               '6',
               '7',
               '8',
               '9',
               '10',
               'J',
               'Q',
               'K',
               'A'],
              'suits': ['spades', 'diamonds', 'clubs', 'hearts']})

In [43]:
dir(mydeck.__class__)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'ranks',
 'suits']